In [1]:
from conv import *
from fcn import *
from client import *

import numpy as np
import pandas as pd
import torch as torch
import torch.nn.functional as F
import crypten

import matplotlib.pyplot as plt

torch.set_printoptions(precision=16)
torch.set_default_tensor_type(torch.DoubleTensor)

from tqdm import tqdm, trange
import time
from torch.utils.data import DataLoader

import warnings
warnings.filterwarnings("ignore")

In [2]:
input_batch = 2
input_channel = 10
input_size = 128
filter_channel = 10
filter_size = 3
act = 'ReLU'
pooling_size = 2

conv_list = []
conv_list.append(Conv(input_batch, 3, input_size, filter_channel, filter_size, act, pooling_size))
input_size = (int)(input_size / 2)

In [3]:
while input_size > 1:
    conv_list.append(Conv(input_batch, input_channel, input_size, filter_channel, filter_size, act, pooling_size))
    input_size = (int)(input_size / 2)

fcn_list = []
output_size = 16

fcn1 = FCN(input_batch, filter_channel, output_size)
fcn_list.append(fcn1)

while output_size > 2:
    fcn_list.append(FCN(input_batch, output_size, (int)(output_size / 2)))
    output_size = (int)(output_size / 2)

In [4]:
def accuracy_f(predictions, client):
    MSE = client.mse_cal(predictions)
    return MSE
    
def train(data):
    hidden_list = []
    hidden_list.append(conv_list[0].forward(data))
    
    for i in range(1, len(conv_list)):
        temp = conv_list[i].forward(hidden_list[-1])
        hidden_list.append(temp)

    hidden_list[-1] = hidden_list[-1].reshape(input_batch, 1, filter_channel)
        
    for fcn_layer in fcn_list:
        temp = fcn_layer.forward(hidden_list[-1])
        hidden_list.append(temp)
        
    return hidden_list[-1]

In [5]:
clients = []
test_clients = []
accs = []
loss_hist = []
for i in range(30):
    client = Client(i + 1)
    clients.append(client)

for i in range(20):
    client = Client(30 + i + 1)
    test_clients.append(client)

Reading /home/dhz/bci-data/ds003478-download/sub-001/eeg/sub-001_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-002/eeg/sub-002_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-003/eeg/sub-003_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-004/eeg/sub-004_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-005/eeg/sub-005_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-006/eeg/sub-006_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-007/eeg/sub-007_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-008/eeg/sub-008_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-009/eeg/sub-009_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-010/eeg/sub-010_task-Rest_run-01_eeg.fdt
Reading /home/dhz/bci-data/ds003478-download/sub-011/eeg/sub-011_task-Rest_run-01_eeg.fdt
Reading /h

In [6]:
ite = 100
for epoch in range(ite):
    losses = 0.
    step = 0
    accuracy = 0.
    mses = 0.

    loop = tqdm(range(len(clients)))
    loop.set_description(f'Epoch [{epoch + 1}/{ite}]')

    for i in loop:
        step += 1
        client = clients[i]

        data = client.get_data()

        prediction = train(data)

        if client.acc(prediction):
            accuracy += 1.

        temp = client

        loss = temp.loss(prediction)

        learning_rate = 1e-2

        prev = client.loss_grad()

        for i in range(len(fcn_list)):
            weight, prev = fcn_list[len(fcn_list) - 1 - i].backpropagation(prev, client, learning_rate)

        prev = prev.reshape(input_batch, filter_channel, 1, 1)

        for i in range(len(conv_list)):
            weight, prev = conv_list[len(conv_list) - 1 - i].backpropagation(prev, client, learning_rate)

        losses = losses + loss.item()
        if step % 30 == 0:
            for c in test_clients:
                p = train(c.get_data())
                mses += accuracy_f(p, c)
            loop.set_postfix(loss = losses / step, mse = mses / 20)

    if epoch % 5 == 0:
        accs.append(mses / step)
        loss_hist.append(losses / step)

Epoch [1/100]:  13%|█████████▏                                                           | 4/30 [00:08<00:57,  2.21s/it]


KeyboardInterrupt: 

In [7]:
client0 = clients[0]

In [8]:
data = client0.get_data()

In [9]:
data.shape

torch.Size([2, 3, 128, 128])